# Explore kaggle data and combine to a single dataset

Run the following notebook to build a combined dataset that will be used for training

In [ ]:
# imports
import pandas as pd

In [ ]:
df1 = pd.read_csv("../data/raw/data_scientist_united_states_job_postings_jobspikr.csv")
df2 = pd.read_csv("../data/raw/data/Uncleaned_DS_jobs.csv")
df3 = pd.read_csv("../data/raw/data/DataScientist.csv")

In [ ]:
df1["job_board"].unique()

In [ ]:
print(df1.head())
print(df1.shape)


In [ ]:
print(df2.head())
print(df2.shape)

In [ ]:
print(df3.head())
print(df3.shape)

Looking at the shape of each of the data sources, we expect the total number of job posts in the combined dataset to be close to 14581.

In [ ]:
# Shows total number of missing values for each column
print(df1.isna().sum())
print(df2.isna().sum())
print(df3.isna().sum())

Looking at the head of the data we see quite a few missing values. We see that in df1, 9 columns contain many missing values. However, this is ok because the columns we are interested in are mainly job_title, company name, and job description. These columns contain almost no missing values which is good.

In [ ]:
# Drop the 1 row where company name is missing
df1 = df1.dropna(subset=["company_name"])
df1 = df1.fillna("Unknown")
print(df1.shape)

Now lets clean up the company name column for df2 and df3, since it looks as if the rating is attached to the end of the company name.

In [ ]:
# clean up the company name by removing the rating attached to the end
df2["Company Name"] = df2["Company Name"].apply(lambda x: x.split('\n')[0])
df3["Company Name"] = df3["Company Name"].apply(lambda x: x.split('\n')[0])

print(df2["Company Name"])
print(df3["Company Name"])

Now we can combine the 3 data sources by job title, company name, job description

In [ ]:
# select only the columns we want
subset1 = df1[["job_title","company_name","job_description","job_board"]]
subset1.columns = ["Job Title", "Company Name","Job Description","Job Board"]

subset2 = df2[["Job Title", "Company Name","Job Description"]]
subset2["Job Board"] = "Glassdoor"
subset3 = df3[["Job Title", "Company Name","Job Description"]]
subset3["Job Board"] = "Glassdoor"

In [ ]:
# Combine all 3 data sources
combined_dataset = pd.concat([subset1,subset2,subset3])

In [ ]:
combined_dataset.head()

In [ ]:
combined_dataset.shape

The shape of the combined dataset matches what we expect to have. So in total we have 14580 data science related job postings.

Let's see the distribution of where the different job posts came from

In [ ]:

combined_dataset["Job Board"].value_counts().plot(kind='pie',figsize=(15,15),autopct='%1.1f%%')

Indeed and Glassdoor make up a major of this dataset. This will be important later on in testing so we can properly split the data.

We have build a combined dataset that will be used for training.

# Explore the Job Descriptions

In [ ]:
# Pick a random job post
# check out 30,
num = 25
job_description = combined_dataset["Job Description"][num].values[0]
job_title = combined_dataset["Job Title"][num].values[0]
print(job_title)
print(job_description)

In [ ]:
# Finding the word count for each job description
combined_dataset['Count'] = combined_dataset['Job Description'].str.split().str.len()

In [ ]:
# Show descriptive statistics abouth the word count
print("Descriptive statistics")
print(f"Min length: {combined_dataset['Count'].describe()}")

In [ ]:
print(combined_dataset['Job Description'][combined_dataset["Count"] == combined_dataset["Count"].min()])
print(combined_dataset['Job Description'][combined_dataset["Count"] == combined_dataset["Count"].max()])

By finding the word count for each of the job descriptions we can better understand how long job description's usually are. The data contains rows where the job description is 2 words, and the longest is 4764 words.

In [ ]:
combined_dataset["Count"].plot.hist(bins=30)

In [ ]:
combined_dataset.boxplot(column=["Count"])